<a href="https://colab.research.google.com/github/mibassan/master/blob/master/Ligthgbm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import pandas as pd

# Lee el archivo como un DataFrame
#df_ventas = pd.read_csv('/Users/micaelabassan/Desktop/fundamentos/Labo 3/sell-in.txt', delimiter='\t',  decimal='.')  # Cambia el delimitador si es necesario
#df_predict = pd.read_csv('/Users/micaelabassan/Desktop/fundamentos/Labo 3/productos_a_predecir.txt', delimiter='\t')  # Cambia el delimitador si es necesario
#df_product = pd.read_csv('/Users/micaelabassan/Desktop/fundamentos/Labo 3/tb_productoscorregida.txt', delimiter='\t')  # Cambia el delimitador si es necesario
#df_stocks = pd.read_csv('/Users/micaelabassan/Desktop/fundamentos/Labo 3/tb_stocks.txt', delimiter='\t')  # Cambia el delimitador si es necesario


In [3]:
import pandas as pd

# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [4]:

# Leer el archivo CSV con pandas
df_ventas = pd.read_csv('/content/drive/My Drive/Labo3/sell-in.txt', delimiter='\t',  decimal='.')  # Cambia el delimitador si es necesario
df_predict = pd.read_csv('/content/drive/My Drive/Labo3/productos_a_predecir.txt', delimiter='\t')  # Cambia el delimitador si es necesario
df_product = pd.read_csv('/content/drive/My Drive/Labo3/tb_productoscorregida.txt', delimiter='\t')  # Cambia el delimitador si es necesario
df_stocks = pd.read_csv('/content/drive/My Drive/Labo3/tb_stocks.txt', delimiter='\t')  # Cambia el delimitador si es necesario

In [18]:
print(df_ventas.head())
print(df_stocks.head())
print(df_product.head())

   periodo  customer_id  product_id  plan_precios_cuidados  cust_request_qty  \
0   201701        10234       20524                      0                 2   
1   201701        10032       20524                      0                 1   
2   201701        10217       20524                      0                 1   
3   201701        10125       20524                      0                 1   
4   201701        10012       20524                      0                11   

   cust_request_tn       tn  
0          0.05300  0.05300  
1          0.13628  0.13628  
2          0.03028  0.03028  
3          0.02271  0.02271  
4          1.54452  1.54452  
   periodo  product_id  stock_final
0   201810       20524      1.61267
1   201810       20311      2.93657
2   201810       20654      6.83269
3   201810       21005      1.01338
4   201810       20974      0.34595
  cat1         cat2     cat3    brand  sku_size  product_id
0   HC  ROPA LAVADO  Liquido  LIMPIEX       900       20280
1  

In [5]:
df_ventas.periodo = pd.to_datetime(df_ventas.periodo, format='%Y%m')
# Convertir 'periodo' a formato de fecha
#data['periodo'] = pd.to_datetime(data['periodo'], format='%Y%m')

In [6]:
df_ventas.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,2017-01-01,10234,20524,0,2,0.05300,0.05300
1,2017-01-01,10032,20524,0,1,0.13628,0.13628
2,2017-01-01,10217,20524,0,1,0.03028,0.03028
3,2017-01-01,10125,20524,0,1,0.02271,0.02271
4,2017-01-01,10012,20524,0,11,1.54452,1.54452


# Armado del dataset para los productos a predecir

In [7]:
# Armado de la lista de productos validos a predecir para el periodo
product_ids_validos = df_predict['product_id'].tolist()

In [8]:
# Filtrar el DataFrame original para quedarse solo con los product_id válidos
# En un nuevo Data Frame llamado "DATA"
data = df_ventas[df_ventas['product_id'].isin(product_ids_validos)]

In [26]:
data.head()

,product_id,periodo,cust_request_qty,customer_id,tn,tn_lag_1,tn_lag_2,tn_lag_3,tn_lag_6,tn_lag_12,year,month
0,20001,2017-01-01,1,10014.0,0,0.0,0.0,0.0,0.0,0.0,2017,1
1,20001,2017-01-01,1,10018.0,1,0.0,0.0,0.0,0.0,0.0,2017,1
2,20001,2017-01-01,1,10024.0,0,1.0,0.0,0.0,0.0,0.0,2017,1
3,20001,2017-01-01,1,10042.0,0,0.0,1.0,0.0,0.0,0.0,2017,1
4,20001,2017-01-01,1,10043.0,0,0.0,0.0,1.0,0.0,0.0,2017,1


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2293481 entries, 0 to 2945817
Data columns (total 7 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   periodo                datetime64[ns]
 1   customer_id            int64         
 2   product_id             int64         
 3   plan_precios_cuidados  int64         
 4   cust_request_qty       int64         
 5   cust_request_tn        float64       
 6   tn                     float64       
dtypes: datetime64[ns](1), float64(2), int64(4)
memory usage: 140.0 MB


In [11]:
# Función para agregar los valores de 'tn' por 'product_id' y 'periodo'

# Definir la función aggregate_data
def aggregate_data(df_ventas):
    return df_ventas.groupby(['product_id', 'periodo','cust_request_qty'])['tn'].sum().reset_index()

# Agrupar los datos
data = aggregate_data(data)

In [26]:
data.head()

,product_id,periodo,cust_request_qty,tn
0,20001,2017-01-01,1,71.89266
1,20001,2017-01-01,2,50.26777
2,20001,2017-01-01,3,73.13506
3,20001,2017-01-01,4,24.64697
4,20001,2017-01-01,5,14.97624


In [12]:
import pandas as pd
import numpy as np

# Obtener todos los productos y periodos únicos
all_product_ids = data['product_id'].unique()
all_periods = data['periodo'].unique()

# Crear un DataFrame con todas las combinaciones posibles de product_id y periodo
all_combinations = pd.MultiIndex.from_product([all_product_ids, all_periods], names=['product_id', 'periodo']).to_frame(index=False)

# Realizar el merge con el DataFrame original, llenando los valores faltantes con 0
df_complete = pd.merge(all_combinations, df_ventas, on=['product_id', 'periodo'], how='left').fillna(0)

# Verificar el resultado
print(df_complete)

# Asegurarse de que los tipos de datos sean correctos
df_complete['cust_request_qty'] = df_complete['cust_request_qty'].astype(int)
df_complete['tn'] = df_complete['tn'].astype(int)

# Mostrar el DataFrame completo
print("\nDataFrame con todos los product_id en todos los periodos:")
print(df_complete)


         product_id    periodo  customer_id  plan_precios_cuidados  \
0             20001 2017-01-01      10001.0                    0.0   
1             20001 2017-01-01      10063.0                    0.0   
2             20001 2017-01-01      10080.0                    0.0   
3             20001 2017-01-01      10094.0                    0.0   
4             20001 2017-01-01      10184.0                    0.0   
...             ...        ...          ...                    ...   
2299207       21276 2019-11-01      10154.0                    0.0   
2299208       21276 2019-12-01      10219.0                    0.0   
2299209       21276 2019-12-01      10052.0                    0.0   
2299210       21276 2019-12-01      10029.0                    0.0   
2299211       21276 2019-12-01      10289.0                    0.0   

         cust_request_qty  cust_request_tn        tn  
0                    11.0         99.43861  99.43861  
1                     1.0          0.12312   0.12

In [13]:
# Contar la cantidad de valores únicos en la columna 'product_id'
unique_product_ids = df_complete['product_id'].nunique()

# Mostrar el resultado
print("Cantidad de valores únicos en 'product_id':", unique_product_ids)


Cantidad de valores únicos en 'product_id': 780


In [14]:
# Función para agregar los valores de 'tn' por 'product_id' y 'periodo'

# Definir la función aggregate_data
def aggregate_data(df_complete):
    return df_complete.groupby(['product_id', 'periodo','cust_request_qty','customer_id'])['tn'].sum().reset_index()

# Agrupar los datos
data2 = aggregate_data(df_complete)

In [15]:
print(data2)

         product_id    periodo  cust_request_qty  customer_id  tn
0             20001 2017-01-01                 1      10014.0   0
1             20001 2017-01-01                 1      10018.0   1
2             20001 2017-01-01                 1      10024.0   0
3             20001 2017-01-01                 1      10042.0   0
4             20001 2017-01-01                 1      10043.0   0
...             ...        ...               ...          ...  ..
2299207       21276 2019-11-01                 2      10550.0   0
2299208       21276 2019-12-01                 1      10029.0   0
2299209       21276 2019-12-01                 1      10052.0   0
2299210       21276 2019-12-01                 1      10219.0   0
2299211       21276 2019-12-01                 1      10289.0   0

[2299212 rows x 5 columns]


# Calulamos **LAGs**

In [17]:
# Agregar lags para la característica 'tn' (variable objetivo)
lags = [1, 2, 3, 6, 12]  # Lags de 1 mes, 2 meses y 3 m

# Agregar lags para 'tn' en el conjunto de datos de data
for lag in lags:
    data2[f'tn_lag_{lag}'] = data2.groupby('product_id')['tn'].shift(lag)
# Reemplazar NaN por 0 en las columnas de lags
lag_columns = [f'tn_lag_{lag}' for lag in lags]
data2[lag_columns] = data2[lag_columns].fillna(0)


In [18]:
print(data2)

         product_id    periodo  cust_request_qty  customer_id  tn  tn_lag_1  \
0             20001 2017-01-01                 1      10014.0   0       0.0   
1             20001 2017-01-01                 1      10018.0   1       0.0   
2             20001 2017-01-01                 1      10024.0   0       1.0   
3             20001 2017-01-01                 1      10042.0   0       0.0   
4             20001 2017-01-01                 1      10043.0   0       0.0   
...             ...        ...               ...          ...  ..       ...   
2299207       21276 2019-11-01                 2      10550.0   0       0.0   
2299208       21276 2019-12-01                 1      10029.0   0       0.0   
2299209       21276 2019-12-01                 1      10052.0   0       0.0   
2299210       21276 2019-12-01                 1      10219.0   0       0.0   
2299211       21276 2019-12-01                 1      10289.0   0       0.0   

         tn_lag_2  tn_lag_3  tn_lag_6  tn_lag_12  


In [19]:
data=data2
print(data)

         product_id    periodo  cust_request_qty  customer_id  tn  tn_lag_1  \
0             20001 2017-01-01                 1      10014.0   0       0.0   
1             20001 2017-01-01                 1      10018.0   1       0.0   
2             20001 2017-01-01                 1      10024.0   0       1.0   
3             20001 2017-01-01                 1      10042.0   0       0.0   
4             20001 2017-01-01                 1      10043.0   0       0.0   
...             ...        ...               ...          ...  ..       ...   
2299207       21276 2019-11-01                 2      10550.0   0       0.0   
2299208       21276 2019-12-01                 1      10029.0   0       0.0   
2299209       21276 2019-12-01                 1      10052.0   0       0.0   
2299210       21276 2019-12-01                 1      10219.0   0       0.0   
2299211       21276 2019-12-01                 1      10289.0   0       0.0   

         tn_lag_2  tn_lag_3  tn_lag_6  tn_lag_12  


# LightGBM **Primer Version**

In [68]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split

data = data2

# Crear características adicionales (features)
data['year'] = data['periodo'].dt.year
data['month'] = data['periodo'].dt.month

# Filtrar los datos hasta 2019-12-01 para entrenamiento
train_data = data[data['periodo'] <= '2019-12-01']

# Definir las características y el target
features = ['product_id', 'customer_id', 'year', 'month', 'cust_request_qty']
target = 'tn'

X = train_data[features]
y = train_data[target]

# Dividir en conjuntos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo LightGBM
model = LGBMRegressor()
model.fit(X_train, y_train, eval_set=[(X_val, y_val)])

# Crear el DataFrame para las predicciones de febrero de 2020
# Usamos todas las combinaciones únicas de product_id y customer_id en el dataset
unique_product_ids = data['product_id'].unique()
unique_customer_ids = data['customer_id'].unique()
prediction_data = pd.DataFrame([(pid, cid) for pid in unique_product_ids for cid in unique_customer_ids], columns=['product_id', 'customer_id'])

# Añadir las columnas necesarias
prediction_data['year'] = 2020
prediction_data['month'] = 2

# Para 'cust_request_qty', usamos el promedio de solicitudes por combinación de product_id y customer_id
cust_request_qty_means = data.groupby(['product_id', 'customer_id'])['cust_request_qty'].mean().reset_index()
prediction_data = prediction_data.merge(cust_request_qty_means, on=['product_id', 'customer_id'], how='left')

# Rellenar NaNs en 'cust_request_qty' con 0 si no hay datos históricos para esa combinación
prediction_data['cust_request_qty'] = prediction_data['cust_request_qty'].fillna(0)

# Asignar valores de 'tn'
prediction_data['tn'] = data['tn']

# Agregar lags para 'tn' en el conjunto de datos de predicción
lags = [1, 2, 3, 6, 12]
for lag in lags:
    prediction_data[f'tn_lag_{lag}'] = prediction_data.groupby('product_id')['tn'].shift(lag)

# Hacer predicciones
X_pred = prediction_data[features]
prediction_data['tn'] = model.predict(X_pred)

# Agrupar por product_id y sumar las predicciones de tn
grouped_predictions = prediction_data.groupby('product_id')['tn'].sum().reset_index()

# Añadir la columna de periodo
grouped_predictions['periodo'] = '202002'

# Seleccionar columnas y guardar en CSV
output = grouped_predictions[['product_id', 'tn']]
output.to_csv('predicciones_feb_2020.csv', index=False)

print("Predicciones guardadas en 'predicciones_feb_2020.csv'")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.031235 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 589
[LightGBM] [Info] Number of data points in the train set: 1839369, number of used features: 5
[LightGBM] [Info] Start training from score 0.352071
Predicciones guardadas en 'predicciones_feb_2020.csv'


In [64]:
print(data)

         product_id    periodo  cust_request_qty  customer_id  tn  year  \
0             20001 2017-01-01                 1      10014.0   0  2017   
1             20001 2017-01-01                 1      10018.0   1  2017   
2             20001 2017-01-01                 1      10024.0   0  2017   
3             20001 2017-01-01                 1      10042.0   0  2017   
4             20001 2017-01-01                 1      10043.0   0  2017   
...             ...        ...               ...          ...  ..   ...   
2299207       21276 2019-11-01                 2      10550.0   0  2019   
2299208       21276 2019-12-01                 1      10029.0   0  2019   
2299209       21276 2019-12-01                 1      10052.0   0  2019   
2299210       21276 2019-12-01                 1      10219.0   0  2019   
2299211       21276 2019-12-01                 1      10289.0   0  2019   

         month  tn_lag_1  tn_lag_2  tn_lag_3  tn_lag_6  tn_lag_12  
0            1       0.0       

# LightGBM CON **LAGS**

In [75]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split

data = data2

# Crear características adicionales (features)
data['year'] = data['periodo'].dt.year
data['month'] = data['periodo'].dt.month

# Agregar lags para 'tn' en el conjunto de datos
lags = [1, 2, 3, 6, 12]
for lag in lags:
    data[f'tn_lag_{lag}'] = data.groupby('product_id')['tn'].shift(lag)

# Filtrar los datos hasta 2019-12-01 para entrenamiento
train_data = data[data['periodo'] <= '2019-12-01']

# Definir las características y el target
features = ['product_id', 'customer_id', 'year', 'month', 'cust_request_qty'] + [f'tn_lag_{lag}' for lag in lags]
target = 'tn'

X = train_data[features]
y = train_data[target]

# Dividir en conjuntos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo LightGBM
model = LGBMRegressor()
model.fit(X_train, y_train, eval_set=[(X_val, y_val)])

# Crear el DataFrame para las predicciones de febrero de 2020
# Usamos todas las combinaciones únicas de product_id y customer_id en el dataset original
unique_product_ids = data2['product_id'].unique()
unique_customer_ids = data2['customer_id'].unique()
prediction_data = pd.DataFrame([(pid, cid) for pid in unique_product_ids for cid in unique_customer_ids], columns=['product_id', 'customer_id'])
# Añadir las columnas necesarias para las características adicionales
prediction_data['year'] = 2020
prediction_data['month'] = 2
# Agregar la columna 'tn' al DataFrame de predicciones
prediction_data['tn'] = np.nan  # Creamos la columna con NaN

# Agregar lags para 'tn' en el conjunto de datos de predicción
for lag in lags:
    prediction_data[f'tn_lag_{lag}'] = prediction_data.groupby('product_id')['tn'].shift(lag)

# Para 'cust_request_qty', usamos el promedio de solicitudes por combinación de product_id y customer_id
cust_request_qty_means = data.groupby(['product_id', 'customer_id'])['cust_request_qty'].mean().reset_index()
prediction_data = prediction_data.merge(cust_request_qty_means, on=['product_id', 'customer_id'], how='left')

# Rellenar NaNs en 'cust_request_qty' con 0 si no hay datos históricos para esa combinación
prediction_data['cust_request_qty'] = prediction_data['cust_request_qty'].fillna(0)

# Hacer predicciones
X_pred = prediction_data[features]
prediction_data['tn'] = model.predict(X_pred)

# Agrupar por product_id y sumar las predicciones de tn
grouped_predictions = prediction_data.groupby('product_id')['tn'].sum().reset_index()

# Añadir la columna de periodo
grouped_predictions['periodo'] = '202002'

# Seleccionar columnas y guardar en CSV
output = grouped_predictions[['product_id', 'tn']]
output.to_csv('predicciones_feb_2020.csv', index=False)

print("Predicciones guardadas en 'predicciones_feb_2020.csv'")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.130672 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1005
[LightGBM] [Info] Number of data points in the train set: 1839369, number of used features: 10
[LightGBM] [Info] Start training from score 0.352071
Predicciones guardadas en 'predicciones_feb_2020.csv'


In [78]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split

data = data2

# Crear características adicionales (features)
data['year'] = data['periodo'].dt.year
data['month'] = data['periodo'].dt.month

# Agregar lags para 'tn' en el conjunto de datos
lags = [1, 2, 3, 6, 12]
for lag in lags:
    data[f'tn_lag_{lag}'] = data.groupby('product_id')['tn'].shift(lag)

# Filtrar los datos hasta 2019-12-01 para entrenamiento
train_data = data[data['periodo'] <= '2019-12-01']

# Definir las características y el target
features = ['product_id', 'customer_id', 'year', 'month', 'cust_request_qty'] + [f'tn_lag_{lag}' for lag in lags]
target = 'tn'

X = train_data[features]
y = train_data[target]

# Reemplazar valores faltantes con 0
X.fillna(0, inplace=True)
y.fillna(0, inplace=True)

# Entrenar un modelo de regresión lineal para cada product_id
product_ids = data['product_id'].unique()
regressions = {}
for pid in product_ids:
    X_pid = X[X['product_id'] == pid]
    y_pid = y[X['product_id'] == pid]
    regression = LinearRegression()
    regression.fit(X_pid, y_pid)
    regressions[pid] = regression

# Crear el DataFrame para las predicciones de febrero de 2020
# Usamos todas las combinaciones únicas de product_id y customer_id en el dataset original
unique_product_ids = data2['product_id'].unique()
unique_customer_ids = data2['customer_id'].unique()
prediction_data = pd.DataFrame([(pid, cid) for pid in unique_product_ids for cid in unique_customer_ids], columns=['product_id', 'customer_id'])
# Añadir las columnas necesarias para las características adicionales
prediction_data['year'] = 2020
prediction_data['month'] = 2
# Agregar la columna 'tn' al DataFrame de predicciones
prediction_data['tn'] = np.nan  # Creamos la columna con NaN

# Agregar lags para 'tn' en el conjunto de datos de predicción
for lag in lags:
    prediction_data[f'tn_lag_{lag}'] = prediction_data.groupby('product_id')['tn'].shift(lag)

# Para 'cust_request_qty', usamos el promedio de solicitudes por combinación de product_id y customer_id
cust_request_qty_means = data.groupby(['product_id', 'customer_id'])['cust_request_qty'].mean().reset_index()
prediction_data = prediction_data.merge(cust_request_qty_means, on=['product_id', 'customer_id'], how='left')

# Rellenar NaNs en 'cust_request_qty' con 0 si no hay datos históricos para esa combinación
prediction_data['cust_request_qty'] = prediction_data['cust_request_qty'].fillna(0)

# Reemplazar valores faltantes con 0
prediction_data.fillna(0, inplace=True)

# Predicciones usando regresión lineal para cada product_id
for pid, regression in regressions.items():
    X_pid_pred = prediction_data[prediction_data['product_id'] == pid][features]
    prediction_data.loc[prediction_data['product_id'] == pid, 'tn'] = regression.predict(X_pid_pred)

# Hacer predicciones finales usando LightGBM
X_pred = prediction_data[features]
prediction_data['tn'] = model.predict(X_pred)

# Operación inversa para obtener los resultados correctos
for pid, regression in regressions.items():
    X_pid_pred = prediction_data[prediction_data['product_id'] == pid][features]
    prediction_data.loc[prediction_data['product_id'] == pid, 'tn'] -= regression.predict(X_pid_pred)

# Agrupar por product_id y sumar las predicciones de tn
grouped_predictions = prediction_data.groupby('product_id')['tn'].sum().reset_index()

# Añadir la columna de periodo
grouped_predictions['periodo'] = '202002'

# Seleccionar columnas y guardar en CSV
output = grouped_predictions[['product_id', 'tn']]
output.to_csv('predicciones_feb_2020.csv', index=False)

print("Predicciones guardadas en 'predicciones_feb_2020.csv'")


<ipython-input-78-8cd773806014>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(0, inplace=True)


Predicciones guardadas en 'predicciones_feb_2020.csv'


# LightGBM con **Regresion lineal**

In [25]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split

data = data2.copy()

# Crear características adicionales (features)
data['year'] = data['periodo'].dt.year
data['month'] = data['periodo'].dt.month

# Agregar lags para 'tn' en el conjunto de datos
lags = [1, 2, 3, 6, 12]
for lag in lags:
    data[f'tn_lag_{lag}'] = data.groupby('product_id')['tn'].shift(lag)

# Reemplazar valores faltantes con 0
data.fillna(0, inplace=True)

# Filtrar los datos hasta 2019-12-01 para entrenamiento
train_data = data[data['periodo'] <= '2019-12-01']

# Definir las características y el target
features = ['product_id', 'customer_id', 'year', 'month', 'cust_request_qty'] + [f'tn_lag_{lag}' for lag in lags]
target = 'tn'

X = train_data[features]
y = train_data[target]

# Dividir en conjuntos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar un modelo de regresión lineal para cada product_id
product_ids = data['product_id'].unique()
regressions = {}
coefs = {}
for pid in product_ids:
    X_pid = X[X['product_id'] == pid]
    y_pid = y[X['product_id'] == pid]
    regression = LinearRegression()
    regression.fit(X_pid, y_pid)
    regressions[pid] = regression
    coefs[pid] = (regression.coef_[0], regression.intercept_)

# Crear el DataFrame para las predicciones de febrero de 2020
unique_product_ids = data2['product_id'].unique()
unique_customer_ids = data2['customer_id'].unique()
prediction_data = pd.DataFrame([(pid, cid) for pid in unique_product_ids for cid in unique_customer_ids], columns=['product_id', 'customer_id'])

# Añadir las columnas necesarias para las características adicionales
prediction_data['year'] = 2020
prediction_data['month'] = 2

# Para 'cust_request_qty', usamos el promedio de solicitudes por combinación de product_id y customer_id
cust_request_qty_means = data.groupby(['product_id', 'customer_id'])['cust_request_qty'].mean().reset_index()
prediction_data = prediction_data.merge(cust_request_qty_means, on=['product_id', 'customer_id'], how='left')

# Reemplazar valores faltantes con 0
prediction_data.fillna(0, inplace=True)

# Agregar lags para 'tn' en el conjunto de datos de predicción
for lag in lags:
    lag_col = f'tn_lag_{lag}'
    prediction_data[lag_col] = 0
    for pid in unique_product_ids:
        mask = prediction_data['product_id'] == pid
        if lag_col in data.columns:
            data_lags = data.loc[data['product_id'] == pid, lag_col].values
            pred_lags = prediction_data.loc[mask, lag_col]
            prediction_data.loc[mask, lag_col] = data_lags[-len(pred_lags):] if len(data_lags) >= len(pred_lags) else np.pad(data_lags, (len(pred_lags) - len(data_lags), 0), 'constant')

# Predicciones usando regresión lineal para cada product_id
for pid, regression in regressions.items():
    X_pid_pred = prediction_data[prediction_data['product_id'] == pid][features]
    prediction_data.loc[prediction_data['product_id'] == pid, 'tn'] = regression.predict(X_pid_pred)

# Hacer predicciones finales usando LightGBM
model = LGBMRegressor()
model.fit(X_train, y_train, eval_set=[(X_val, y_val)])
X_pred = prediction_data[features]
prediction_data['transformed_tn'] = model.predict(X_pred)

# Invertir la transformación para obtener los valores originales
for pid, (coef, intercept) in coefs.items():
    mask = prediction_data['product_id'] == pid
    prediction_data.loc[mask, 'tn'] = prediction_data.loc[mask, 'transformed_tn'] * coef + intercept

# Agrupar por product_id y sumar las predicciones de tn
grouped_predictions = prediction_data.groupby('product_id')['tn'].sum().reset_index()

# Añadir la columna de periodo
grouped_predictions['periodo'] = '202002'

# Seleccionar columnas y guardar en CSV
output = grouped_predictions[['product_id', 'tn']]
output.to_csv('predicciones_feb_2020.csv', index=False)

print("Predicciones guardadas en 'predicciones_feb_2020.csv'")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.211673 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1000
[LightGBM] [Info] Number of data points in the train set: 1839369, number of used features: 10
[LightGBM] [Info] Start training from score 0.352071
Predicciones guardadas en 'predicciones_feb_2020.csv'


# LigthGBM normalizando

In [28]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler

data = data2.copy()

# Crear características adicionales (features)
data['year'] = data['periodo'].dt.year
data['month'] = data['periodo'].dt.month

# Agregar lags para 'tn' en el conjunto de datos
lags = [1, 2, 3, 6, 12]
for lag in lags:
    data[f'tn_lag_{lag}'] = data.groupby('product_id')['tn'].shift(lag)

# Reemplazar valores faltantes con 0
data.fillna(0, inplace=True)

# Filtrar los datos hasta 2019-12-01 para entrenamiento
train_data = data[data['periodo'] <= '2019-12-01']

# Definir las características y el target
features = ['product_id','customer_id','year', 'month', 'cust_request_qty'] + [f'tn_lag_{lag}' for lag in lags]
target = 'tn'

X = train_data[features]
y = train_data[target]

# Normalizar características
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Dividir en conjuntos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar un modelo de regresión lineal para cada product_id
product_ids = data['product_id'].unique()
regressions = {}
coefs = {}
for pid in product_ids:
    X_pid = train_data[train_data['product_id'] == pid][features]
    y_pid = train_data[train_data['product_id'] == pid][target]
    X_pid = scaler.fit_transform(X_pid)
    regression = LinearRegression()
    regression.fit(X_pid, y_pid)
    regressions[pid] = regression
    coefs[pid] = (regression.coef_, regression.intercept_)

# Crear el DataFrame para las predicciones de febrero de 2020
unique_product_ids = data2['product_id'].unique()
unique_customer_ids = data2['customer_id'].unique()
prediction_data = pd.DataFrame([(pid, cid) for pid in unique_product_ids for cid in unique_customer_ids], columns=['product_id', 'customer_id'])

# Añadir las columnas necesarias para las características adicionales
prediction_data['year'] = 2020
prediction_data['month'] = 2

# Para 'cust_request_qty', usamos el promedio de solicitudes por combinación de product_id y customer_id
cust_request_qty_means = data.groupby(['product_id', 'customer_id'])['cust_request_qty'].mean().reset_index()
prediction_data = prediction_data.merge(cust_request_qty_means, on=['product_id', 'customer_id'], how='left')

# Reemplazar valores faltantes con 0
prediction_data.fillna(0, inplace=True)

# Agregar lags para 'tn' en el conjunto de datos de predicción
for lag in lags:
    lag_col = f'tn_lag_{lag}'
    prediction_data[lag_col] = 0
    for pid in unique_product_ids:
        mask = prediction_data['product_id'] == pid
        if lag_col in data.columns:
            data_lags = data.loc[data['product_id'] == pid, lag_col].values
            pred_lags = prediction_data.loc[mask, lag_col]
            prediction_data.loc[mask, lag_col] = data_lags[-len(pred_lags):] if len(data_lags) >= len(pred_lags) else np.pad(data_lags, (len(pred_lags) - len(data_lags), 0), 'constant')

# Predicciones usando regresión lineal para cada product_id
for pid, regression in regressions.items():
    X_pid_pred = prediction_data[prediction_data['product_id'] == pid][features]
    X_pid_pred = scaler.transform(X_pid_pred)
    prediction_data.loc[prediction_data['product_id'] == pid, 'tn'] = regression.predict(X_pid_pred)

# Ajustar los hiperparámetros del modelo LightGBM
param_grid = {
    'num_leaves': [31, 50],
    'min_data_in_leaf': [20, 50, 100],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 500]
}
lgb_model = LGBMRegressor()
grid = GridSearchCV(lgb_model, param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1)
grid.fit(X_train, y_train)

# Hacer predicciones finales usando el mejor modelo LightGBM
best_model = grid.best_estimator_
X_pred = scaler.transform(prediction_data[features])
prediction_data['transformed_tn'] = best_model.predict(X_pred)

# Invertir la transformación para obtener los valores originales
for pid, (coef, intercept) in coefs.items():
    mask = prediction_data['product_id'] == pid
    prediction_data.loc[mask, 'tn'] = prediction_data.loc[mask, 'transformed_tn'] * coef[0] + intercept

# Agrupar por product_id y sumar las predicciones de tn
grouped_predictions = prediction_data.groupby('product_id')['tn'].sum().reset_index()

# Añadir la columna de periodo
grouped_predictions['periodo'] = '202002'

# Seleccionar columnas y guardar en CSV
output = grouped_predictions[['product_id', 'tn']]
output.to_csv('predicciones_feb_2020.csv', index=False)

print("Predicciones guardadas en 'predicciones_feb_2020.csv'")


Fitting 3 folds for each of 54 candidates, totalling 162 fits
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.105458 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1030
[LightGBM] [Info] Number of data points in the train set: 1226246, number of used features: 10
[LightGBM] [Info] Start training from score 0.352096
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is s